In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

La funzione detect_anomalies Restituisce le entry del dataframe che hanno almeno <seconds_threshold> secondi di valori uguali. 

In [ ]:
def detect_anomalies_old(accelerometer_df: pd.DataFrame, hand_type: str, seconds_threshold: int = 5) -> pd.DataFrame:
    """
    Restituisce le entry del dataframe che hanno almeno <seconds_threshold> secondi di valori uguali
    hand_type: D -> Mano dominante, ND -> Mano non dominante
    """
    if hand_type != "D" and hand_type != "ND":
        raise ValueError("hand_type deve essere 'D' o 'ND'")
    
    campionamenti_per_secondo = 8 # l'accelerometro esegue 8 campionamenti al secondo
    seconds_threshold = seconds_threshold*campionamenti_per_secondo

    df_diff = accelerometer_df[f'x_{hand_type}', f'y_{hand_type}', f'z_{hand_type}'].diff()
    anomalies = df_diff.rolling(window=seconds_threshold).apply(lambda x: all(num == 0 for num in x)) # Recupera le righe che hanno almeno 5 secondi di valori uguali
    anomaly_rows = accelerometer_df[anomalies.any(axis=1)]
    return anomaly_rows

In [ ]:
df = pd.read_csv("../raw_data/1_week_RAW.csv")

In [ ]:
df1 = pd.read_csv("../raw_data/1_week_RAW.csv", nrows=691200, skiprows=lambda x: x in [1, 691200])

In [ ]:
df2 = pd.read_csv("../raw_data/1_week_RAW.csv", nrows=691200, skiprows=lambda x: x in [1, 691200*2])

In [ ]:
df.describe()

In [ ]:
df1.describe()

In [ ]:
plt.figure(figsize=(18,6))
plt.plot(df['x_ND'])

plt.show()

# Add a comment here to describe the code and the work done ASAP



In [ ]:
def detect_anomalies(accelerometer_df: pd.DataFrame, hand_type: str, seconds_threshold: int = 28800) -> pd.DataFrame:
    """
    Restituisce le entry del dataframe che hanno almeno <seconds_threshold> secondi di valori uguali
    hand_type: D -> Mano dominante, ND -> Mano non dominante
    """
    if hand_type != "D" and hand_type != "ND":
        raise ValueError("hand_type deve essere 'D' o 'ND'")
    
    df_diff = accelerometer_df[[f'x_{hand_type}', f'y_{hand_type}', f'z_{hand_type}']].diff()
    anomalies = df_diff.rolling(window=seconds_threshold).apply(lambda x: all(abs(num) <= 0.05 for num in x)) # Recupera le righe che hanno almeno 5 secondi di valori uguali
    anomaly_rows = accelerometer_df[anomalies.any(axis=1)]
    return anomaly_rows

In [ ]:

indexes = []

for i in range(0, 24):
    if i != 0:
        if (df2["x_D"].iloc[691200*i:691200*(i+1)].std()<0.01) and (df2["y_D"].iloc[691200*i:691200*(i+1)].std()<0.01) and(df2["z_D"].iloc[691200*i:691200*(i+1)].std()<0.01) and(df2["x_ND"].iloc[691200*i:691200*(i+1)].std()<0.01) and(df2["y_ND"].iloc[691200*i:691200*(i+1)].std()<0.01) and (df2["z_ND"].iloc[691200*i:691200*(i+1)].std()<0.01):
            indexes.append(i)
    else:
        if (df2["x_D"].iloc[i:691200].std()<0.01) and (df2["y_D"].iloc[i:691200].std()<0.01) and (df2["z_D"].iloc[i:691200].std()<0.01) and (df2["x_ND"].iloc[i:691200].std()<0.01) and (df2["y_ND"].iloc[i:691200].std()<0.01) and (df2["z_ND"].iloc[i:691200].std()<0.01):
            indexes.append(i)

# Describe above code: ?????
print(indexes)

In [ ]:
# This should do the samw as the code above

indexes_rewritten = []
HOURS = 24
SAMPLE_RATE = 8
TIME_INTERVAL = 60*60*SAMPLE_RATE

columns = ["x_D", "y_D", "z_D", "x_ND", "y_ND", "z_ND"]

for i in range(HOURS):
    start = i * TIME_INTERVAL
    end = (i + 1) * TIME_INTERVAL if i != 0 else TIME_INTERVAL
    if all(df2[col].iloc[start:end].std() < 0.01 for col in columns):
        indexes_rewritten.append(i)
        
print(indexes_rewritten)

In [ ]:
indexes2 = []
for i in range(0,24):
    if i != 0:
        if (df1["x_D"].iloc[691200*i:691200*(i+1)].std()<0.05) and (df1["y_D"].iloc[691200*i:691200*(i+1)].std()<0.05) and (df1["z_D"].iloc[691200*i:691200*(i+1)].std()<0.05) and (df1["x_ND"].iloc[691200*i:691200*(i+1)].std()<0.05) and (df1["y_ND"].iloc[691200*i:691200*(i+1)].std()<0.05) and (df1["z_ND"].iloc[691200*i:691200*(i+1)].std()<0.05):
            indexes2.append(i)
    else:
        if (df1["x_D"].iloc[i:691200].std()<0.05) and (df1["y_D"].iloc[i:691200].std()<0.05) and (df1["z_D"].iloc[i:691200].std()<0.05) and (df1["x_ND"].iloc[i:691200].std()<0.05) and (df1["y_ND"].iloc[i:691200].std()<0.05) and (df1["z_ND"].iloc[i:691200].std()<0.01):
            indexes2.append(i)

print(indexes2)

In [ ]:
# This should do the samw as the code above

indexes2_rewritten = []
HOURS = 24
SAMPLE_RATE = 8
TIME_INTERVAL = 60*60*SAMPLE_RATE
columns = ["x_D", "y_D", "z_D", "x_ND", "y_ND", "z_ND"]

for i in range(HOURS):
    start = i * TIME_INTERVAL
    end = (i + 1) * TIME_INTERVAL if i != 0 else TIME_INTERVAL
    if all(df1[col].iloc[start:end].std() < 0.05 for col in columns):
        indexes2_rewritten.append(i) 

In [ ]:
detect_anomalies(df1, "ND", 28800)

In [ ]:
magnitude_D = np.sqrt(np.square(df['x_D']) + np.square(df['y_D']) + np.square(df['z_D'])) #mano dominante
magnitude_ND = np.sqrt(np.square(df['x_ND']) + np.square(df['y_ND']) + np.square(df['z_ND'])) #mano non dominante

In [ ]:
plt.figure(figsize=(30,20))
plt.plot(magnitude_D)

plt.show()